In [10]:
from spyglass.common import Session
from hex_maze_behavior import HexMazeBlock, HexMazeChoice

import pandas as pd

In [34]:
# Join HexMazeChoice with the Session table so we can get session_id and subject_id
# Get Trial attributes too like opto_cond
df = pd.DataFrame((HexMazeChoice() * HexMazeBlock.Trial() * Session()).fetch(as_dict=True))
display(df)

df["epoch_id"] = df["session_id"].astype(str) + "_epoch" + df["epoch"].astype(str)
df["choose_left"] = (df["choice_direction"] == "left").astype(int)

# Open question: in Tim's paper, random effects were estimated over the levels of rat and session-within-rat
# For Berke Lab, we only have one session (epoch) per day. BUT for Frank lab, we have multiple epochs per session.
# Do we want to estimate random effects over the level of epoch-within-session, or epoch-within-rat??

# Keep only columns needed for regression
regression_cols = [
    "subject_id",            # random effect
    "session_id",            # nested grouping/random effect
    "epoch_id",              # nested grouping/random effect
    "session_description",   # Helpful for opto (Frank Lab)
    "block",                 # we may want to filter based on block?
    "block_trial_num",       # we may want to filter to only trials >25 to allow for learning
    "opto_cond",             # we may want to filter based on opto condition to see effects here
    "choose_left",           # dependent variable
    "reward_prob_diff",      # predictor
    "path_length_diff",      # predictor
]
regression_df = df[regression_cols].copy()

# Convert all non-numeric columns to 'category' dtype
for col in regression_df.columns:
    if not pd.api.types.is_numeric_dtype(regression_df[col]):
        regression_df[col] = regression_df[col].astype('str')

display(regression_df.dtypes)
display(regression_df)


,nwb_file_name,epoch,block,block_trial_num,choice_direction,reward_prob_diff,path_length_diff,interval_list_name,epoch_trial_num,reward,...,poke_interval,duration,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
0,BraveLu20240519_.nwb,1,1,2,left,-80.0,-8.0,epoch1_block1_trial2,2,0,...,"[1716147484.063145, 1716147493.082886]",22.47300,BraveLu,"University of California, San Francisco",Loren Frank,BraveLu_20240519,RippleStimDelayNoDelay,2024-05-19 11:56:55,1970-01-01,Hexmaze Bandit
1,BraveLu20240519_.nwb,1,1,3,right,-40.0,-2.0,epoch1_block1_trial3,3,1,...,"[1716147502.0209603, 1716147514.247437]",21.16460,BraveLu,"University of California, San Francisco",Loren Frank,BraveLu_20240519,RippleStimDelayNoDelay,2024-05-19 11:56:55,1970-01-01,Hexmaze Bandit
2,BraveLu20240519_.nwb,1,1,4,left,-80.0,-8.0,epoch1_block1_trial4,4,1,...,"[1716147522.8466442, 1716147536.0942888]",21.84690,BraveLu,"University of California, San Francisco",Loren Frank,BraveLu_20240519,RippleStimDelayNoDelay,2024-05-19 11:56:55,1970-01-01,Hexmaze Bandit
3,BraveLu20240519_.nwb,1,1,5,right,-40.0,-2.0,epoch1_block1_trial5,5,0,...,"[1716147542.415394, 1716147543.8906953]",7.79641,BraveLu,"University of California, San Francisco",Loren Frank,BraveLu_20240519,RippleStimDelayNoDelay,2024-05-19 11:56:55,1970-01-01,Hexmaze Bandit
4,BraveLu20240519_.nwb,1,1,6,left,-80.0,-8.0,epoch1_block1_trial6,6,0,...,"[1716147550.0813005, 1716147551.0693681]",7.17867,BraveLu,"University of California, San Francisco",Loren Frank,BraveLu_20240519,RippleStimDelayNoDelay,2024-05-19 11:56:55,1970-01-01,Hexmaze Bandit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5096,Toby20250318_.nwb,7,5,14,right,40.0,2.0,epoch7_block5_trial14,212,0,...,"[1742343226.993721, 1742343228.2196705]",8.06141,Toby,"University of California, San Francisco",Loren Frank,Toby_20250318,HexmazeNoStim,2025-03-18 10:24:08,1970-01-01,Hexmaze Bandit
5097,Toby20250318_.nwb,7,5,15,left,80.0,0.0,epoch7_block5_trial15,213,1,...,"[1742343233.5560741, 1742343241.1129408]",12.89330,Toby,"University of California, San Francisco",Loren Frank,Toby_20250318,HexmazeNoStim,2025-03-18 10:24:08,1970-01-01,Hexmaze Bandit
5098,Toby20250318_.nwb,7,5,16,right,40.0,2.0,epoch7_block5_trial16,214,1,...,"[1742343246.4812782, 1742343254.5434513]",13.43050,Toby,"University of California, San Francisco",Loren Frank,Toby_20250318,HexmazeNoStim,2025-03-18 10:24:08,1970-01-01,Hexmaze Bandit
5099,Toby20250318_.nwb,7,5,17,left,80.0,0.0,epoch7_block5_trial17,215,1,...,"[1742343260.567464, 1742343269.0050755]",14.46160,Toby,"University of California, San Francisco",Loren Frank,Toby_20250318,HexmazeNoStim,2025-03-18 10:24:08,1970-01-01,Hexmaze Bandit


subject_id              object
session_id              object
epoch_id                object
session_description     object
block                    int64
block_trial_num          int64
opto_cond               object
choose_left              int64
reward_prob_diff       float64
path_length_diff       float64
dtype: object

,subject_id,session_id,epoch_id,session_description,block,block_trial_num,opto_cond,choose_left,reward_prob_diff,path_length_diff
0,BraveLu,BraveLu_20240519,BraveLu_20240519_epoch1,RippleStimDelayNoDelay,1,2,None,1,-80.0,-8.0
1,BraveLu,BraveLu_20240519,BraveLu_20240519_epoch1,RippleStimDelayNoDelay,1,3,delay,0,-40.0,-2.0
2,BraveLu,BraveLu_20240519,BraveLu_20240519_epoch1,RippleStimDelayNoDelay,1,4,delay,1,-80.0,-8.0
3,BraveLu,BraveLu_20240519,BraveLu_20240519_epoch1,RippleStimDelayNoDelay,1,5,None,0,-40.0,-2.0
4,BraveLu,BraveLu_20240519,BraveLu_20240519_epoch1,RippleStimDelayNoDelay,1,6,None,1,-80.0,-8.0
...,...,...,...,...,...,...,...,...,...,...
5096,Toby,Toby_20250318,Toby_20250318_epoch7,HexmazeNoStim,5,14,None,0,40.0,2.0
5097,Toby,Toby_20250318,Toby_20250318_epoch7,HexmazeNoStim,5,15,None,1,80.0,0.0
5098,Toby,Toby_20250318,Toby_20250318_epoch7,HexmazeNoStim,5,16,None,0,40.0,2.0
5099,Toby,Toby_20250318,Toby_20250318_epoch7,HexmazeNoStim,5,17,None,1,80.0,0.0


In [39]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pymer4.models import lmer

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri


# Scale predictors
scaler = MinMaxScaler()
regression_df[['reward_prob_diff_scaled', 'path_length_diff_scaled']] = scaler.fit_transform(regression_df[['reward_prob_diff', 'path_length_diff']])

# Subset trials > 25
df_subset = regression_df[regression_df['block_trial_num'] > 25]

essential_cols = ["choose_left", "reward_prob_diff_scaled", "path_length_diff_scaled", "subject_id", "session_id"]
df_subset = df_subset[essential_cols].copy().reset_index(drop=True)

# Mixed logit model (identical to Tim's glmer)
model = lmer(
    formula=(
        "choose_left ~ reward_prob_diff_scaled + path_length_diff_scaled "
        "+ (reward_prob_diff_scaled + path_length_diff_scaled | subject_id) "
        "+ (reward_prob_diff_scaled + path_length_diff_scaled | session_id)"
    ),
    data=df_subset,
    family='binomial'
)

print(model)
results = model.fit()
print(results)

# Save outputs
results.coefs.to_csv("chooseLregSummary.csv")
model.ranef.to_csv("chooseLregCoefsByRat.csv")

pymer4.models.lmer(fitted=False, formula=choose_left~reward_prob_diff_scaled+path_length_diff_scaled+(reward_prob_diff_scaled+path_length_diff_scaled|subject_id)+(reward_prob_diff_scaled+path_length_diff_scaled|session_id), family=binomial, link=default)


NotImplementedError: Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>'

In [43]:
import pandas as pd
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from sklearn.preprocessing import MinMaxScaler

# Import required R packages
base = importr('base')
lme4 = importr('lme4')

# Scale predictors
scaler = MinMaxScaler()
regression_df[['reward_prob_diff_scaled', 'path_length_diff_scaled']] = scaler.fit_transform(
    regression_df[['reward_prob_diff', 'path_length_diff']]
)

# Subset trials > 25
df_subset = regression_df[regression_df['block_trial_num'] > 25].copy()
    
# Convert and assign to R - keep conversion inside the context
with localconverter(pandas2ri.converter):
    r_df = pandas2ri.py2rpy(df_subset)

# Now assign outside the converter (r_df is already an R object)
r.assign('df_r', r_df)

# Fit model in R
r('''
model <- glmer(
    choose_left ~ reward_prob_diff_scaled + path_length_diff_scaled +
    (reward_prob_diff_scaled + path_length_diff_scaled | subject_id) +
    (reward_prob_diff_scaled + path_length_diff_scaled | session_id),
    data = df_r,
    family = binomial,
    control = glmerControl(optimizer = "bobyqa")
)
''')

# Print model summary
print(r('summary(model)'))

# Extract fixed effects coefficients
r('''
coef_summary <- as.data.frame(summary(model)$coefficients)
coef_summary$term <- rownames(coef_summary)
''')

# Extract random effects by subject
r('''
ranef_subject <- as.data.frame(coef(model)$subject_id)
ranef_subject$subject_id <- rownames(ranef_subject)
''')

# Save outputs
r('write.csv(coef_summary, "chooseLregSummary.csv", row.names = FALSE)')
r('write.csv(ranef_subject, "chooseLregCoefsByRat.csv", row.names = FALSE)')

# Get the results back into Python
with localconverter(pandas2ri.converter):
    coef_summary = pandas2ri.rpy2py(r['coef_summary'])
    ranef_subject = pandas2ri.rpy2py(r['ranef_subject'])

print("\nFixed Effects:")
print(coef_summary)
print("\nRandom Effects by Subject:")
print(ranef_subject)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: choose_left ~ reward_prob_diff_scaled + path_length_diff_scaled +  
    (reward_prob_diff_scaled + path_length_diff_scaled | subject_id) +  
    (reward_prob_diff_scaled + path_length_diff_scaled | session_id)
   Data: df_r
Control: glmerControl(optimizer = "bobyqa")

      AIC       BIC    logLik -2*log(L)  df.resid 
   3929.8    4019.3   -1949.9    3899.8      2867 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1460 -0.9455 -0.6129  0.9976  2.6293 

Random effects:
 Groups     Name                    Variance Std.Dev. Corr       
 session_id (Intercept)             0.51779  0.7196              
            reward_prob_diff_scaled 1.10136  1.0495   -0.92      
            path_length_diff_scaled 0.29189  0.5403    0.35 -0.69
 subject_id (Intercept)             0.29226  0.5406              
            reward_prob_diff_scaled 0.09497  0.3

NotImplementedError: Conversion 'rpy2py' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>'